# Reflect Tables into SQLAlchemy ORM

### Import dependencies and sqlalchemy toolkit

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Column, Integer, String, Float, Date

### Read cleaned CSV files

In [2]:
# Read "clean_hawaii_measurements.csv" file
measurements = pd.read_csv("Resources/clean_hawaii_measurements.csv")

# Reformat the date column to year-month-date
measurements["date"] = pd.to_datetime(measurements["date"], format="%Y-%m-%d")

# View the first several rows of the "clean_hawaii_measurements.csv" file
measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
# Read "clean_hawaii_stations.csv" file
stations = pd.read_csv("Resources/clean_hawaii_stations.csv")

# View the first several rows of the "clean_hawaii_stations.csv" file
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


### View dtypes for CSV files

In [4]:
# View dtypes for measurements table
measurements.dtypes

station            object
date       datetime64[ns]
prcp              float64
tobs                int64
dtype: object

In [5]:
# View dtypes for stations table
stations.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

### Use the engine and connect to database hawaii.sqlite

In [6]:
# Connect engine to hawaii.sqlite file
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

### Reflect the existing database

In [7]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

### View the classes

In [8]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

### Save references

In [9]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

### Create sessions

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

### Inspect tables

In [11]:
# Get table names
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [12]:
# Inspect `measurement` table datatypes
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [13]:
# Inspect `station` table datatypes
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


### Confirm table creation

In [14]:
engine.execute("select * from measurement limit 10;").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (10, 'USC00519397', '2010-01-11', 0.01, 64.0)]

In [15]:
engine.execute("select * from station limit 10;").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]